In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading 

In [194]:
df = pd.read_excel("Online Retail.xlsx")
print("Number of rows in the dataset: ", df.shape[0])
print("Number of columns in the dataset: ", df.shape[1])

Number of rows in the dataset:  541909
Number of columns in the dataset:  8


In [195]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Data Inspection & Data Cleaning

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [198]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [199]:
cancelled = df[df['InvoiceNo'].astype(str).str.startswith('C')]
negative_price = df[df['UnitPrice'] < 0]
negative_quantity = df[df['Quantity'] < 0]
print("Number of cancelled orders: ", cancelled.shape[0])
print("Number of orders with price corrections: ", negative_price.shape[0])
print("Number of rows with negative 'Quantity': ", negative_quantity.shape[0])

Number of cancelled orders:  9288
Number of orders with price corrections:  2
Number of rows with negative 'Quantity':  10624


The dataset contains negative values for both Quantity and UnitPrice. Negative values for Quantity likely represent canceled orders, which we can identify by checking if InvoiceNo starts with the letter 'C'. These transactions are separated into a new dataframe for deeper insights, such as analyzing patterns in cancellations, frequent reasons, or products that are often returned.

Negative values for UnitPrice might indicate corrections or anomalies in pricing, which could distort the analysis. These entries have been removed to ensure the integrity and accuracy of our findings, as retaining them could bias metrics like average sales or trends.

In [201]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

Lastly, we check for the number of duplicated orders and remove any missing values. 

In [203]:
print("Number of duplicated orders:", len(df[df.duplicated()]))

Number of duplicated orders: 5226


In [204]:
df.drop_duplicates(inplace = True)
df = df.dropna()

In [205]:
print("Number of rows in the dataset: ", df.shape[0])

Number of rows in the dataset:  392692


In [206]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

## Feature Engineering

In [208]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

#### Calculating Total Sales

In [213]:
df['TotalSales'] = df['Quantity'] * df['UnitPrice']

#### Adding Hour, Day, Month, and Year Features 

In [215]:
df['Hour'] = df['InvoiceDate'].dt.hour
df['Day'] = df['InvoiceDate'].dt.day_name()
df['InvoiceMonth'] = df['InvoiceDate'].dt.month
df['InvoiceYear']  = df['InvoiceDate'].dt.year
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSales,TotalRevenue,Hour,Day,InvoiceMonth,InvoiceYear
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,15.30,8,Wednesday,12,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20.34,8,Wednesday,12,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,22.00,8,Wednesday,12,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20.34,8,Wednesday,12,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,20.34,8,Wednesday,12,2010


## Explanatory Data Analysis (EDA)

#### Monthly Sales Trend

#### Top 10 Products by Quantity Sold (2010 vs 2011)